In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import sys
import multiprocessing as mp
from datetime import datetime
from bs4 import BeautifulSoup
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
tqdm.pandas()
import time
import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
import matplotlib.pyplot as plt
import itertools
import networkx as nx
#from pyvis.network import Network

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn import metrics


import numpy as np

#import pygraphviz
import pandas as pd
#import holoviews as hv
import networkx as nx


import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from gensim.corpora.dictionary import Dictionary
#from pyemd import emd
from numpy import dot, float32 as REAL, memmap as np_memmap, \
    double, array, zeros, vstack, sqrt, newaxis, integer, \
    ndarray,prod, argmax

import logging
from gensim.models.phrases import Phraser
import itertools
from functools import partial

## modules
sys.path.insert(0, './_modules')
from w2v_functions import text2ngram, wmd_eval
from knn_kmeans_functions import cluster_func, knn_func, cluster_predict, cluster_func_dbscan
from general_functions import corpus_tool
from path_definitions import control_dir, w2v_dir, unigrams_dir, bigrams_dir, output_dir

plt.style.use('ggplot')


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/home/dev/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/dev/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
## text iterator/streaming
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield [token for token in line.split()] # line.split()




In [4]:
cwd = os.getcwd()

In [5]:
date_tag = str(datetime.today().date())
print (date_tag)

2020-06-06


In [6]:
## related to the project name:
name_tag = 'sigo'

In [7]:
cores = mp.cpu_count()
print (cores)
partitions = cores

4


### Defining paths:

In [8]:
## specifying the path to collect control:
control_path = control_dir()


Folder already exists.
/home/dev/sigo/_collect_control


In [9]:
## specifying the path to database:
text_path = unigrams_dir()


Folder already exists.
/home/dev/sigo/_text_unigrams


In [10]:
bigrams_path = bigrams_dir()

Folder already exists.
/home/dev/sigo/_text_bigrams


In [11]:
## specifying the path to database:
w2v_path = w2v_dir()


Folder already exists.
/home/dev/sigo/_w2v_data


In [12]:
output_path = output_dir()

Folder already exists.
/home/dev/sigo/_data_output


In [13]:
df = pd.read_csv(os.path.join(output_path, 'mail_data_clean.csv'), sep ='\t', encoding = 'utf-8')

In [14]:
df

,main_id,mail_id,thread_id,mail_date_received,mail_hms_received,mail_subject,mail_from,mail_body,mail_body_snippet,has_attachment,attachment_folder_link,case_id,external_ref_number,date_created,date_updated,mail_datetime_received,mail_body_snippet_clean,mail_subject_clean,mail_snippet+subject_clean
0,1,16e621b40e2a0718,16e3d0dba31c9f02,2019-11-13 00:00:00,00:13:02,Re: RES: [0-9150000028251] Referente ao Pedido...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,NaN,0-9150000028251,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-13 00:13:02,hi attached response letter case production sh...,re res referente pedido dra helena,re res referente pedido dra helena hi attached...
1,2,16d5f62086593562,16d2ce551f3bb65a,2019-09-23 00:00:00,18:28:42,Re: [6-5526000027768] - URGENT - 24 HOURS - OV...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,NaN,6-5526000027768,0000904-07.2019.8.17.0260,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-09-23 18:28:42,hi attached response letter case production sh...,urgent hours overdue september criminal gmail ...,urgent hours overdue september criminal gmail ...
2,3,16c0626420c86643,16bfb3ebc5852120,2019-07-18 00:00:00,17:34:13,Re: (TAÍS PLS READ) [1-0673000027027] 2019/07/...,tatesser@google.com,"b""Oi Vitor,\r\n\r\nComo falamos, vamos produzi...","Oi Vitor, Como falamos, vamos produzir os dado...",0,NaN,1-0673000027027,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-07-18 17:34:13,oi vitor falamos vamos produzir dados cadastra...,tais pls read criminal authority google play a...,tais pls read criminal authority google play a...
3,4,16eb93bc25e0f323,16eb93bc25e0f323,2019-11-29 00:00:00,22:15:31,FWD: [5-0366000028367] OFICIO 007900-8/2019,lis-latam@google.com,"b'\r\n\r\nUrgent - Fines\r\n\r\nHi team,\r\n\r...","Urgent - Fines Hi team, Please see below the r...",1,NaN,5-0366000028367,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-29 22:15:31,urgent fines hi team please see request lis re...,fwd oficio,fwd oficio urgent fines hi team please see req...
4,5,1700ca8ed5948627,16fe9671f13a5e14,2020-02-03 00:00:00,20:06:01,Re: [0-5710000029462] - LEGAL ISSUE - c 2020/0...,lis-latam@google.com,"b'\r\n\r\nHi all,\r\n\r\n\r\nAttached the resp...","Hi all, Attached the response letter for this ...",1,NaN,0-5710000029462,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2020-02-03 20:06:01,hi attached response letter case production sh...,legal issue february criminal gmail embezzleme...,legal issue february criminal gmail embezzleme...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18984,18985,17287a461441595a,17287a461441595a,2020-06-06 00:00:00,03:19:55,FWD: [7-3186000030160] BR Data Request 1368918...,lis-latam@google.com,b'\r\n\r\nUrgent - 5 day deadline\r\n\r\nRecor...,Urgent - 5 day deadline Records # [1368918-59....,1,NaN,7-3186000030160,1368918-59.2020.8.13.0702,2020-06-06 03:22:49,2020-06-06 03:22:49,2020-06-06 03:19:55,urgent day deadline records date received via ...,fwd br data request,fwd br data request urgent day deadline record...
18985,18986,17287b98c01dc43e,17287b98c01dc43e,2020-06-06 00:00:00,03:43:03,FWD: [7-8996000030704] BR Data Request 406/2019,lis-latam@google.com,b'\r\n\r\nUrgent\r\n\r\nRecords # [0000457-52....,Urgent Records # [0000457-52.2019.815.0461] Da...,1,NaN,7-8996000030704,0000457-52.2019.815.0461,2020-06-06 03:45:53,2020-06-06 03:45:53,2020-06-06 03:43:03,urgent records date received via lers june ler...,fwd br data request,fwd br data request urgent records date receiv...
18986,18987,17287ca82a1ccb47,17287ca82a1ccb47,2020-06-06 00:00:00,04:01:33,FWD: [5-3357000031005] BR Data Request 0032023...,lis-latam@google.com,b'\r\n\r\nRecords # [0032023-88.2019.8.19.0014...,Records # [0032023-88.2019.8.19.0014] Date rec...,1,NaN,5-3357000031005,0032023-88.2019.8.19.0014,2020-06-06 04:06:14,2020-06-0

In [15]:
df['mail_body_snippet_clean'] = df['mail_body_snippet_clean'].fillna('')

### Loading the iterator:

In [16]:
## loading the iterator:
sentences = MySentences(text_path)  # a memory-friendly iterator


### Training bigrams:

In [17]:
os.listdir(w2v_path)

['trained_bigrams_sigo_2020-04-01',
 'trained_bigrams_sigo_2020-03-01',
 'trained_bigrams_sigo_2020-03-03',
 'word2vec_model_sigo_2020-02-28',
 'trained_bigrams_sigo_2020-02-28',
 'word2vec_model_sigo_2020-02-26',
 'trained_bigrams_sigo_2020-02-29',
 'trained_bigrams_sigo_2020-02-26',
 'trained_bigrams_sigo_2020-03-26',
 'word2vec_model_sigo_2020-03-01',
 'word2vec_model_sigo_2020-03-10',
 'word2vec_model_sigo_2020-02-27',
 'word2vec_model_sigo_2020-03-26',
 'trained_bigrams_sigo_2020-03-06',
 'trained_bigrams_sigo_2020-02-27',
 'trained_bigrams_sigo_2020-03-04',
 'word2vec_model_sigo_2020-03-06',
 'word2vec_model_sigo_2020-04-01',
 'trained_bigrams_sigo_2020-03-10',
 'word2vec_model_sigo_2020-03-04',
 'word2vec_model_sigo_2020-03-03',
 'word2vec_model_sigo_2020-02-29']

In [18]:
training = True

if training == True:
    print('Training bigrams')
    print(' ')
    phrases = gensim.models.Phrases(sentences)
    
    
    filename = 'trained_bigrams_{}_{}'.format(name_tag, date_tag)
    phrases.save(os.path.join(w2v_path, filename)) 
    
    model_bigrams = gensim.models.Phrases.load(os.path.join(w2v_path, filename))
    
else:
    filename = 'trained_bigrams_{}_{}'.format(name_tag, '2019-07-01')
    model_bigrams = gensim.models.Phrases.load(os.path.join(w2v_path, filename))

print('Loading bigrams')
print(' ')
bigram = Phraser(model_bigrams)

2020-06-06 12:29:31,955 : INFO : collecting all words and their counts
2020-06-06 12:29:31,957 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-06-06 12:29:32,272 : INFO : PROGRESS: at sentence #10000, processed 184299 words and 43698 word types


Training bigrams
 


2020-06-06 12:29:32,405 : INFO : collected 54160 word types from a corpus of 260326 words (unigram + bigrams) and 14126 sentences
2020-06-06 12:29:32,407 : INFO : using 54160 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2020-06-06 12:29:32,409 : INFO : saving Phrases object under /home/dev/sigo/_w2v_data/trained_bigrams_sigo_2020-06-06, separately None
2020-06-06 12:29:32,487 : INFO : saved /home/dev/sigo/_w2v_data/trained_bigrams_sigo_2020-06-06
2020-06-06 12:29:32,489 : INFO : loading Phrases object from /home/dev/sigo/_w2v_data/trained_bigrams_sigo_2020-06-06
2020-06-06 12:29:32,539 : INFO : loaded /home/dev/sigo/_w2v_data/trained_bigrams_sigo_2020-06-06
2020-06-06 12:29:32,541 : INFO : source_vocab length 54160


Loading bigrams
 


2020-06-06 12:29:33,015 : INFO : Phraser built with 1742 phrasegrams


In [19]:
bigram['ministerio publico'.split()]

['ministerio_publico']

In [20]:
df.columns

Index(['main_id', 'mail_id', 'thread_id', 'mail_date_received',
       'mail_hms_received', 'mail_subject', 'mail_from', 'mail_body',
       'mail_body_snippet', 'has_attachment', 'attachment_folder_link',
       'case_id', 'external_ref_number', 'date_created', 'date_updated',
       'mail_datetime_received', 'mail_body_snippet_clean',
       'mail_subject_clean', 'mail_snippet+subject_clean'],
      dtype='object')

In [21]:
bigram_func = partial(text2ngram, ngram_model = bigram)

In [22]:
## creating a partial function:
bigram_func = partial(text2ngram, ngram_model = bigram)
## creating a new column containing the text converted to bigrams
df['mail_body_snippet_clean_bigram'] = df['mail_body_snippet_clean'].progress_map(bigram_func)
df['mail_snippet+subject_clean_bigram'] = df['mail_snippet+subject_clean'].progress_map(bigram_func)

100%|██████████| 18989/18989 [00:01<00:00, 17459.49it/s]


In [23]:
df

,main_id,mail_id,thread_id,mail_date_received,mail_hms_received,mail_subject,mail_from,mail_body,mail_body_snippet,has_attachment,...,case_id,external_ref_number,date_created,date_updated,mail_datetime_received,mail_body_snippet_clean,mail_subject_clean,mail_snippet+subject_clean,mail_body_snippet_clean_bigram,mail_snippet+subject_clean_bigram
0,1,16e621b40e2a0718,16e3d0dba31c9f02,2019-11-13 00:00:00,00:13:02,Re: RES: [0-9150000028251] Referente ao Pedido...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,...,0-9150000028251,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-13 00:13:02,hi attached response letter case production sh...,re res referente pedido dra helena,re res referente pedido dra helena hi attached...,hi attached_response letter_case production_sh...,re res referente pedido dra helena hi attached...
1,2,16d5f62086593562,16d2ce551f3bb65a,2019-09-23 00:00:00,18:28:42,Re: [6-5526000027768] - URGENT - 24 HOURS - OV...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,...,6-5526000027768,0000904-07.2019.8.17.0260,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-09-23 18:28:42,hi attached response letter case production sh...,urgent hours overdue september criminal gmail ...,urgent hours overdue september criminal gmail ...,hi attached_response letter_case production_sh...,urgent_hours overdue september criminal_gmail ...
2,3,16c0626420c86643,16bfb3ebc5852120,2019-07-18 00:00:00,17:34:13,Re: (TAÍS PLS READ) [1-0673000027027] 2019/07/...,tatesser@google.com,"b""Oi Vitor,\r\n\r\nComo falamos, vamos produzi...","Oi Vitor, Como falamos, vamos produzir os dado...",0,...,1-0673000027027,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-07-18 17:34:13,oi vitor falamos vamos produzir dados cadastra...,tais pls read criminal authority google play a...,tais pls read criminal authority google play a...,oi vitor falamos vamos produzir dados_cadastra...,tais pls read criminal authority google play a...
3,4,16eb93bc25e0f323,16eb93bc25e0f323,2019-11-29 00:00:00,22:15:31,FWD: [5-0366000028367] OFICIO 007900-8/2019,lis-latam@google.com,"b'\r\n\r\nUrgent - Fines\r\n\r\nHi team,\r\n\r...","Urgent - Fines Hi team, Please see below the r...",1,...,5-0366000028367,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-29 22:15:31,urgent fines hi team please see request lis re...,fwd oficio,fwd oficio urgent fines hi team please see req...,urgent fines hi team_please see_request lis re...,fwd oficio urgent fines hi team_please see_req...
4,5,1700ca8ed5948627,16fe9671f13a5e14,2020-02-03 00:00:00,20:06:01,Re: [0-5710000029462] - LEGAL ISSUE - c 2020/0...,lis-latam@google.com,"b'\r\n\r\nHi all,\r\n\r\n\r\nAttached the resp...","Hi all, Attached the response letter for this ...",1,...,0-5710000029462,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2020-02-03 20:06:01,hi attached response letter case production sh...,legal issue february criminal gmail embezzleme...,legal issue february criminal gmail embezzleme...,hi attached_response letter_case production_sh...,legal_issue february criminal_gmail embezzleme...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18984,18985,17287a461441595a,17287a461441595a,2020-06-06 00:00:00,03:19:55,FWD: [7-3186000030160] BR Data Request 1368918...,lis-latam@google.com,b'\r\n\r\nUrgent - 5 day deadline\r\n\r\nRecor...,Urgent - 5 day deadline Records # [1368918-59....,1,...,7-3186000030160,1368918-59.2020.8.13.0702,2020-06-06 03:22:49,2020-06-06 03:22:49,2020-06-06 03:19:55,urgent day deadline records date received via ...,fwd br data request,fwd br data request urgent day deadline record...,urgent_day deadline_records date received via ...,fwd br data request urgent_day deadline_record...
18985,18986,17287b98c01dc43e,17287b98c01dc43e,2020-06-06 00:00:00,03:43:03,FWD: [7-8996000030704] BR Data Request 406/2019,lis-latam@google.

In [24]:
df['mail_body_snippet_clean_bigram'][6]

'hi attached_response letter_case production_shared directly_le via lers best joseph lis fri_sep mohamad'

In [25]:
## saving above df to file
df.to_csv(os.path.join(output_path, 'mail_data_clean_bigrams.csv'),\
                  sep = '\t', encoding = 'utf-8', index = False)

In [26]:
## saving the text bigrams to file, in order to read it with an iterator
with open(os.path.join(bigrams_path, 'texts_bigrams.dat'), 'w') as f:
    for k in range(0, len(df)):
        f.write(df['mail_body_snippet_clean_bigram'][k] + '\n')
    f.close()

In [27]:
df

,main_id,mail_id,thread_id,mail_date_received,mail_hms_received,mail_subject,mail_from,mail_body,mail_body_snippet,has_attachment,...,case_id,external_ref_number,date_created,date_updated,mail_datetime_received,mail_body_snippet_clean,mail_subject_clean,mail_snippet+subject_clean,mail_body_snippet_clean_bigram,mail_snippet+subject_clean_bigram
0,1,16e621b40e2a0718,16e3d0dba31c9f02,2019-11-13 00:00:00,00:13:02,Re: RES: [0-9150000028251] Referente ao Pedido...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,...,0-9150000028251,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-13 00:13:02,hi attached response letter case production sh...,re res referente pedido dra helena,re res referente pedido dra helena hi attached...,hi attached_response letter_case production_sh...,re res referente pedido dra helena hi attached...
1,2,16d5f62086593562,16d2ce551f3bb65a,2019-09-23 00:00:00,18:28:42,Re: [6-5526000027768] - URGENT - 24 HOURS - OV...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,...,6-5526000027768,0000904-07.2019.8.17.0260,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-09-23 18:28:42,hi attached response letter case production sh...,urgent hours overdue september criminal gmail ...,urgent hours overdue september criminal gmail ...,hi attached_response letter_case production_sh...,urgent_hours overdue september criminal_gmail ...
2,3,16c0626420c86643,16bfb3ebc5852120,2019-07-18 00:00:00,17:34:13,Re: (TAÍS PLS READ) [1-0673000027027] 2019/07/...,tatesser@google.com,"b""Oi Vitor,\r\n\r\nComo falamos, vamos produzi...","Oi Vitor, Como falamos, vamos produzir os dado...",0,...,1-0673000027027,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-07-18 17:34:13,oi vitor falamos vamos produzir dados cadastra...,tais pls read criminal authority google play a...,tais pls read criminal authority google play a...,oi vitor falamos vamos produzir dados_cadastra...,tais pls read criminal authority google play a...
3,4,16eb93bc25e0f323,16eb93bc25e0f323,2019-11-29 00:00:00,22:15:31,FWD: [5-0366000028367] OFICIO 007900-8/2019,lis-latam@google.com,"b'\r\n\r\nUrgent - Fines\r\n\r\nHi team,\r\n\r...","Urgent - Fines Hi team, Please see below the r...",1,...,5-0366000028367,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-29 22:15:31,urgent fines hi team please see request lis re...,fwd oficio,fwd oficio urgent fines hi team please see req...,urgent fines hi team_please see_request lis re...,fwd oficio urgent fines hi team_please see_req...
4,5,1700ca8ed5948627,16fe9671f13a5e14,2020-02-03 00:00:00,20:06:01,Re: [0-5710000029462] - LEGAL ISSUE - c 2020/0...,lis-latam@google.com,"b'\r\n\r\nHi all,\r\n\r\n\r\nAttached the resp...","Hi all, Attached the response letter for this ...",1,...,0-5710000029462,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2020-02-03 20:06:01,hi attached response letter case production sh...,legal issue february criminal gmail embezzleme...,legal issue february criminal gmail embezzleme...,hi attached_response letter_case production_sh...,legal_issue february criminal_gmail embezzleme...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18984,18985,17287a461441595a,17287a461441595a,2020-06-06 00:00:00,03:19:55,FWD: [7-3186000030160] BR Data Request 1368918...,lis-latam@google.com,b'\r\n\r\nUrgent - 5 day deadline\r\n\r\nRecor...,Urgent - 5 day deadline Records # [1368918-59....,1,...,7-3186000030160,1368918-59.2020.8.13.0702,2020-06-06 03:22:49,2020-06-06 03:22:49,2020-06-06 03:19:55,urgent day deadline records date received via ...,fwd br data request,fwd br data request urgent day deadline record...,urgent_day deadline_records date received via ...,fwd br data request urgent_day deadline_record...
18985,18986,17287b98c01dc43e,17287b98c01dc43e,2020-06-06 00:00:00,03:43:03,FWD: [7-8996000030704] BR Data Request 406/2019,lis-latam@google.

### Training the word2vec model:

In [28]:
sentences2 = MySentences(bigrams_path)

In [29]:
filename = 'word2vec_model_{}_{}'.format(name_tag, date_tag)
fname = get_tmpfile(os.path.join(w2v_path, filename))

training = True
if training == True:
    ## training
    model = gensim.models.Word2Vec(sentences2, size=500, window=5, min_count=3, workers=4, iter = 50)
    filename = 'word2vec_model_{}_{}'.format(name_tag, date_tag)
    
    model.save(fname)    
    
else:
    ## loading model if already trained
    model = gensim.models.Word2Vec.load(fname)    
    
    

2020-06-06 12:29:37,771 : INFO : collecting all words and their counts
2020-06-06 12:29:37,772 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-06 12:29:37,821 : INFO : PROGRESS: at sentence #10000, processed 135660 words, keeping 7352 word types
2020-06-06 12:29:37,866 : INFO : collected 9554 word types from a corpus of 259256 raw words and 18989 sentences
2020-06-06 12:29:37,867 : INFO : Loading a fresh vocabulary
2020-06-06 12:29:37,880 : INFO : effective_min_count=3 retains 5090 unique words (53% of original 9554, drops 4464)
2020-06-06 12:29:37,881 : INFO : effective_min_count=3 leaves 253538 word corpus (97% of original 259256, drops 5718)
2020-06-06 12:29:37,897 : INFO : deleting the raw counts dictionary of 9554 items
2020-06-06 12:29:37,898 : INFO : sample=0.001 downsamples 52 most-common words
2020-06-06 12:29:37,899 : INFO : downsampling leaves estimated 173415 word corpus (68.4% of prior 253538)
2020-06-06 12:29:37,911 : INFO : estimated re

2020-06-06 12:29:44,967 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-06 12:29:44,991 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-06 12:29:45,001 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-06 12:29:45,003 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-06 12:29:45,005 : INFO : EPOCH - 15 : training on 259256 raw words (173658 effective words) took 0.4s, 469962 effective words/s
2020-06-06 12:29:45,342 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-06 12:29:45,371 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-06 12:29:45,374 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-06 12:29:45,378 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-06 12:29:45,379 : INFO : EPOCH - 16 : training on 259256 raw words (173614 effective words) took 0.4s, 468670 effective words/s


2020-06-06 12:29:51,206 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-06 12:29:51,207 : INFO : EPOCH - 31 : training on 259256 raw words (173198 effective words) took 0.4s, 467156 effective words/s
2020-06-06 12:29:51,558 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-06 12:29:51,565 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-06 12:29:51,575 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-06 12:29:51,577 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-06 12:29:51,578 : INFO : EPOCH - 32 : training on 259256 raw words (173081 effective words) took 0.4s, 471430 effective words/s
2020-06-06 12:29:51,916 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-06 12:29:51,919 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-06 12:29:51,939 : INFO : worker thread finished; awaiting finish of 1 more threads


2020-06-06 12:29:57,784 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-06 12:29:57,787 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-06 12:29:57,797 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-06 12:29:57,803 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-06 12:29:57,804 : INFO : EPOCH - 48 : training on 259256 raw words (173152 effective words) took 0.5s, 372083 effective words/s
2020-06-06 12:29:58,153 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-06 12:29:58,170 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-06 12:29:58,173 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-06 12:29:58,183 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-06 12:29:58,184 : INFO : EPOCH - 49 : training on 259256 raw words (173345 effective words) took 0.4s, 460821 effective words/s


In [30]:
input_text = 'bom dia'
model.wv[bigram[input_text.split()]]

array([[-4.20594186e-01, -3.57789063e+00, -9.51362312e-01,
        -4.32771593e-01, -1.70192826e+00,  1.47855973e+00,
         1.27962518e+00,  1.43616033e+00, -8.85811806e-01,
         1.00729990e+00, -1.61320424e+00, -8.96929562e-01,
         8.36624026e-01, -1.53869438e+00,  5.64015567e-01,
        -1.83202744e+00, -7.71782339e-01,  6.11842036e-01,
        -2.89113820e-01, -2.12661415e-01,  1.92424953e+00,
         6.96878016e-01, -4.38748091e-01,  1.03540289e+00,
         8.66683424e-02,  1.29384771e-01,  1.93716392e-01,
        -4.55949865e-02,  7.04292059e-02, -6.26431584e-01,
        -9.83848393e-01, -2.26578736e+00, -3.57101649e-01,
         9.42667067e-01,  3.54231596e-01, -1.78554598e-02,
        -1.49673867e+00, -1.78604579e+00, -1.32076859e+00,
         7.57513987e-03, -6.70250058e-01, -1.42572358e-01,
        -6.33795679e-01, -1.47788048e+00, -6.17114723e-01,
         7.81246662e-01, -1.20611799e+00,  4.93504882e-01,
        -8.67020011e-01,  2.15009511e-01, -2.44657308e-0

In [31]:
input_text = 'google'
model.wv.most_similar(bigram[input_text.split()], topn=10)

2020-06-06 12:29:58,985 : INFO : precomputing L2-norms of word weight vectors


[('decision', 0.5136128664016724),
 ('email', 0.493988037109375),
 ('duplicity', 0.48077744245529175),
 ('properly', 0.4686713218688965),
 ('bank', 0.46815580129623413),
 ('items', 0.4609672725200653),
 ('police_request', 0.4551512598991394),
 ('petition', 0.45142853260040283),
 ('advice', 0.4474945068359375),
 ('documents', 0.4448035657405853)]

In [32]:
model.wv.most_similar('ministerio_publico', topn=10)

[('promotor_justica', 0.8013700842857361),
 ('assistente_militar', 0.7936700582504272),
 ('procurador_republica', 0.777875542640686),
 ('ministerio', 0.7561851739883423),
 ('mpu', 0.7553459405899048),
 ('territorios', 0.7516922950744629),
 ('assessor', 0.7514645457267761),
 ('estado_mato', 0.7147400379180908),
 ('tecnico', 0.7073601484298706),
 ('freire', 0.6956809759140015)]

In [33]:
model.wv.most_similar('google', topn=10)

[('decision', 0.5136128664016724),
 ('email', 0.493988037109375),
 ('duplicity', 0.48077744245529175),
 ('properly', 0.4686713218688965),
 ('bank', 0.46815580129623413),
 ('items', 0.4609672725200653),
 ('police_request', 0.4551512598991394),
 ('petition', 0.45142853260040283),
 ('advice', 0.4474945068359375),
 ('documents', 0.4448035657405853)]

In [34]:
model.wv.most_similar('marielle', topn=10)

[('terrorism', 0.6895174980163574),
 ('federal_jfpr', 0.6762751340866089),
 ('day_deadline', 0.6436092853546143),
 ('hour_deadline', 0.6220929622650146),
 ('federal_jfrn', 0.605532169342041),
 ('federal_jfsc', 0.6008020639419556),
 ('kidnapping', 0.5996854305267334),
 ('hours', 0.5961407423019409),
 ('urgent_hours', 0.5920014977455139),
 ('days_deadline', 0.5841942429542542)]

In [35]:
df

,main_id,mail_id,thread_id,mail_date_received,mail_hms_received,mail_subject,mail_from,mail_body,mail_body_snippet,has_attachment,...,case_id,external_ref_number,date_created,date_updated,mail_datetime_received,mail_body_snippet_clean,mail_subject_clean,mail_snippet+subject_clean,mail_body_snippet_clean_bigram,mail_snippet+subject_clean_bigram
0,1,16e621b40e2a0718,16e3d0dba31c9f02,2019-11-13 00:00:00,00:13:02,Re: RES: [0-9150000028251] Referente ao Pedido...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,...,0-9150000028251,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-13 00:13:02,hi attached response letter case production sh...,re res referente pedido dra helena,re res referente pedido dra helena hi attached...,hi attached_response letter_case production_sh...,re res referente pedido dra helena hi attached...
1,2,16d5f62086593562,16d2ce551f3bb65a,2019-09-23 00:00:00,18:28:42,Re: [6-5526000027768] - URGENT - 24 HOURS - OV...,lis-latam@google.com,"b'Hi all,\r\n\r\nAttached the response letter ...","Hi all, Attached the response letter for this ...",1,...,6-5526000027768,0000904-07.2019.8.17.0260,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-09-23 18:28:42,hi attached response letter case production sh...,urgent hours overdue september criminal gmail ...,urgent hours overdue september criminal gmail ...,hi attached_response letter_case production_sh...,urgent_hours overdue september criminal_gmail ...
2,3,16c0626420c86643,16bfb3ebc5852120,2019-07-18 00:00:00,17:34:13,Re: (TAÍS PLS READ) [1-0673000027027] 2019/07/...,tatesser@google.com,"b""Oi Vitor,\r\n\r\nComo falamos, vamos produzi...","Oi Vitor, Como falamos, vamos produzir os dado...",0,...,1-0673000027027,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-07-18 17:34:13,oi vitor falamos vamos produzir dados cadastra...,tais pls read criminal authority google play a...,tais pls read criminal authority google play a...,oi vitor falamos vamos produzir dados_cadastra...,tais pls read criminal authority google play a...
3,4,16eb93bc25e0f323,16eb93bc25e0f323,2019-11-29 00:00:00,22:15:31,FWD: [5-0366000028367] OFICIO 007900-8/2019,lis-latam@google.com,"b'\r\n\r\nUrgent - Fines\r\n\r\nHi team,\r\n\r...","Urgent - Fines Hi team, Please see below the r...",1,...,5-0366000028367,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2019-11-29 22:15:31,urgent fines hi team please see request lis re...,fwd oficio,fwd oficio urgent fines hi team please see req...,urgent fines hi team_please see_request lis re...,fwd oficio urgent fines hi team_please see_req...
4,5,1700ca8ed5948627,16fe9671f13a5e14,2020-02-03 00:00:00,20:06:01,Re: [0-5710000029462] - LEGAL ISSUE - c 2020/0...,lis-latam@google.com,"b'\r\n\r\nHi all,\r\n\r\n\r\nAttached the resp...","Hi all, Attached the response letter for this ...",1,...,0-5710000029462,NaN,2020-03-03 11:18:03,2020-03-03 11:18:03,2020-02-03 20:06:01,hi attached response letter case production sh...,legal issue february criminal gmail embezzleme...,legal issue february criminal gmail embezzleme...,hi attached_response letter_case production_sh...,legal_issue february criminal_gmail embezzleme...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18984,18985,17287a461441595a,17287a461441595a,2020-06-06 00:00:00,03:19:55,FWD: [7-3186000030160] BR Data Request 1368918...,lis-latam@google.com,b'\r\n\r\nUrgent - 5 day deadline\r\n\r\nRecor...,Urgent - 5 day deadline Records # [1368918-59....,1,...,7-3186000030160,1368918-59.2020.8.13.0702,2020-06-06 03:22:49,2020-06-06 03:22:49,2020-06-06 03:19:55,urgent day deadline records date received via ...,fwd br data request,fwd br data request urgent day deadline record...,urgent_day deadline_records date received via ...,fwd br data request urgent_day deadline_record...
18985,18986,17287b98c01dc43e,17287b98c01dc43e,2020-06-06 00:00:00,03:43:03,FWD: [7-8996000030704] BR Data Request 406/2019,lis-latam@google.

In [36]:
d1 = df['mail_body_snippet_clean_bigram'][1]
print(d1)
d2 = df['mail_body_snippet_clean_bigram'][0]
print('----')
print(d2)
print(' ')
print(' ')
dist = wmd_eval(d1, d2, model)
print(dist)

2020-06-06 12:29:59,175 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-06-06 12:29:59,176 : INFO : built Dictionary(14 unique tokens: ['attached_response', 'best_alex', 'directly_le', 'fri_sep', 'hi']...) from 2 documents (total 22 corpus positions)


hi attached_response letter_case production_shared directly_le via lers best_alex lis fri_sep
----
hi attached_response letter_case production_shared directly_le via lers best sybil lis wed_nov helena
 
 
5.272932109054426
